## $\color{Green}{\text{GMD Table 4: GMSL from Workflows}}$

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{Use Workflow data from Bob's run on Amarel}}$

In [38]:
# Load FACTS workflow data as a dataframe.
df_WF = pd.read_fwf('001_IP_names_Modules_Data/mod-submod-data_Bob_amarel_WF.txt')
df_WF

,Component,Workflow,Data_File
0,total,wf1e,total.workflow.wf1e.global.nc
1,total,wf1f,total.workflow.wf1f.global.nc
2,total,wf2e,total.workflow.wf2e.global.nc
3,total,wf2f,total.workflow.wf2f.global.nc


In [39]:
modNO       = df_WF.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];  
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for val in modNO:
    for scenario in SCENARIO: 
        WORKFLOW = df_WF.loc[val].at['Workflow'];  COMP = df_WF["Component"][val];  
        # Skip if data is absent.
        if df_WF.loc[val].at['Data_File'] == 'XXX':
           continue 
        # Pick the data files. 
        dataFOLDER  = '/projects/kopp/facts-experiments/221206/coupling.{arg1}/output/'.format(arg1=scenario)
        dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df_WF.loc[val].at['Data_File']
        dataPATH    = dataFOLDER + dataFILE
        # IMPORT .nc data.
        d_nc = xr.open_dataset(dataPATH)
        #
        #
        # Percentile calculation.
        # percentList = [50, 5, 17, 83, 95]
        percentList = [50, 17, 83]
        #
        # Find year index to pick SLC value
        Yind    = np.where(d_nc["years"].values == 2100)[0][0]
        GMSL = (d_nc["sea_level_change"][:,Yind,0].values)/1000
        #
        # Find Percentile ranges.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        TEMP = [COMP,WORKFLOW, scenario] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Component','Workflow','SSP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
df_ptile
# df_ptile.set_index(['Component','Workflow'])
# df_ptile1 = df_ptile.stack()
# df_ptile1 = df_ptile.unstack()
# df_ptile1




,Component,Workflow,SSP,col_50,col_17,col_83
0,total,wf1e,ssp126,0.40,0.32,0.49
1,total,wf1e,ssp245,0.50,0.42,0.59
2,total,wf1e,ssp585,0.70,0.60,0.80
3,total,wf1f,ssp126,0.41,0.33,0.50
4,total,wf1f,ssp245,0.50,0.41,0.59
5,total,wf1f,ssp585,0.64,0.57,0.75
6,total,wf2e,ssp126,0.46,0.36,0.59
7,total,wf2e,ssp245,0.57,0.47,0.73
8,total,wf2e,ssp585,0.86,0.71,1.07
9,total,wf2f,ssp126,0.47,0.39,0.60


In [72]:
t = df_ptile.set_index( ['Component', 'Workflow', 'SSP'] ).unstack().swaplevel(0,1, axis=1).sort_index(1)
t = t.reindex( ['col_50', 'col_17', 'col_83'], axis=1, level=1  )

#t.loc[:, (slice(None), ['col_50', 'col_17', 'col_83'] )]
t

/tmp/ipykernel_39317/2036077271.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  t = df_ptile.set_index( ['Component', 'Workflow', 'SSP'] ).unstack().swaplevel(0,1, axis=1).sort_index(1)


SSP                ssp126               ssp245               ssp585         \
                   col_50 col_17 col_83 col_50 col_17 col_83 col_50 col_17   
Component Workflow                                                           
total     wf1e       0.40   0.32   0.49   0.50   0.42   0.59   0.70   0.60   
          wf1f       0.41   0.33   0.50   0.50   0.41   0.59   0.64   0.57   
          wf2e       0.46   0.36   0.59   0.57   0.47   0.73   0.86   0.71   
          wf2f       0.47   0.39   0.60   0.58   0.49   0.75   0.79   0.70   

SSP                        
                   col_83  
Component Workflow         
total     wf1e       0.80  
          wf1f       0.75  
          wf2e       1.07  
          wf2f       0.96

In [119]:
df_ptile.loc[:, ["col_50"]]

,col_50
0,0.40
1,0.50
2,0.70
3,0.41
4,0.50
5,0.64
6,0.46
7,0.57
8,0.86
9,0.47


In [128]:
f = lambda x: x*x

def ff(x):
    return x*x

f(5), ff(5)

(25, 25)

In [120]:
df_ptile['stat'] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )

#t = pd.DataFrame( df_ptile.set_index( ['Component', 'Workflow', 'SSP'] )['stat'] ).unstack().swaplevel( 0,1, axis=1 )
t = df_ptile.set_index( ['Component', 'Workflow', 'SSP'] ).loc[:, ['stat']].unstack().swaplevel( 0,1, axis=1 )
t

SSP                             ssp126              ssp245              ssp585
                                  stat                stat                stat
Component Workflow                                                            
total     wf1e      0.40 (0.32 , 0.49)  0.50 (0.42 , 0.59)  0.70 (0.60 , 0.80)
          wf1f      0.41 (0.33 , 0.50)  0.50 (0.41 , 0.59)  0.64 (0.57 , 0.75)
          wf2e      0.46 (0.36 , 0.59)  0.57 (0.47 , 0.73)  0.86 (0.71 , 1.07)
          wf2f      0.47 (0.39 , 0.60)  0.58 (0.49 , 0.75)  0.79 (0.70 , 0.96)

In [121]:
t.to_latex()

/tmp/ipykernel_39317/2828585438.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  t.to_latex()


'\\begin{tabular}{lllll}\n\\toprule\n      & SSP &              ssp126 &              ssp245 &              ssp585 \\\\\n      & {} &                stat &                stat &                stat \\\\\nComponent & Workflow &                     &                     &                     \\\\\n\\midrule\ntotal & wf1e &  0.40 (0.32 , 0.49) &  0.50 (0.42 , 0.59) &  0.70 (0.60 , 0.80) \\\\\n      & wf1f &  0.41 (0.33 , 0.50) &  0.50 (0.41 , 0.59) &  0.64 (0.57 , 0.75) \\\\\n      & wf2e &  0.46 (0.36 , 0.59) &  0.57 (0.47 , 0.73) &  0.86 (0.71 , 1.07) \\\\\n      & wf2f &  0.47 (0.39 , 0.60) &  0.58 (0.49 , 0.75) &  0.79 (0.70 , 0.96) \\\\\n\\bottomrule\n\\end{tabular}\n'

In [115]:
df_ptile.loc[:, ['p1', 'p2']] = df_ptile.apply(lambda x: pd.Series( { 'p1': f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', 'p2': 0} ), axis=1 )

t = df_ptile.set_index( ['Component', 'Workflow', 'SSP'] )[ ['p1', 'p2']].unstack().swaplevel( 0, 1, axis=1)

SSP                             ssp126              ssp245  \
                                    p1                  p1   
Component Workflow                                           
total     wf1e      0.40 (0.32 , 0.49)  0.50 (0.42 , 0.59)   
          wf1f      0.41 (0.33 , 0.50)  0.50 (0.41 , 0.59)   
          wf2e      0.46 (0.36 , 0.59)  0.57 (0.47 , 0.73)   
          wf2f      0.47 (0.39 , 0.60)  0.58 (0.49 , 0.75)   

SSP                             ssp585 ssp126 ssp245 ssp585  
                                    p1     p2     p2     p2  
Component Workflow                                           
total     wf1e      0.70 (0.60 , 0.80)      0      0      0  
          wf1f      0.64 (0.57 , 0.75)      0      0      0  
          wf2e      0.86 (0.71 , 1.07)      0      0      0  
          wf2f      0.79 (0.70 , 0.96)      0      0      0

In [125]:
display( t )

SSP                             ssp126              ssp245              ssp585
                                  stat                stat                stat
Component Workflow                                                            
total     wf1e      0.40 (0.32 , 0.49)  0.50 (0.42 , 0.59)  0.70 (0.60 , 0.80)
          wf1f      0.41 (0.33 , 0.50)  0.50 (0.41 , 0.59)  0.64 (0.57 , 0.75)
          wf2e      0.46 (0.36 , 0.59)  0.57 (0.47 , 0.73)  0.86 (0.71 , 1.07)
          wf2f      0.47 (0.39 , 0.60)  0.58 (0.49 , 0.75)  0.79 (0.70 , 0.96)

In [96]:
df_ptile.groupby( ['Component', 'Workflow', 'SSP'] ).apply(lambda x: pd.Series( { 'p1': f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', 'p2': 0} )  )


TypeError: unsupported format string passed to Series.__format__